In [77]:
import pandas as pd
import pyodbc
from fuzzywuzzy import fuzz
import datetime as dt

In [78]:
username = 'vigrose'
password = 'slytherin10946'

In [79]:
s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
AMADM = pyodbc.connect(s)

In [80]:
ppd = pd.read_csv('../../Data/PPD/ppd_data_20200404.csv')
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
ppd['ME']=fix_me(list(ppd['ME']))

In [81]:
may_phys_query= \
    """ 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    O.ORDER_NBR,
    O.ORDER_PRODUCT_ID, 
    O.ORDER_PHYSICIAN_HIST_KEY,
    O.CUSTOMER_KEY
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND
    D.MONTH_NBR = 5
    AND
    D.YR = 2020
    """

In [82]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

In [83]:
ppd['ME']=fix_me(list(ppd['ME']))

In [84]:
MAY_PHYS = pd.read_sql(con=AMADM, sql=may_phys_query)
MAY_PHYS.head()

,FULL_DT,ME,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY
0,2020-05-06,-1,04660924,4915513,-1,56462
1,2020-05-06,-1,04660935,4915494,-1,56853
2,2020-05-06,-1,04660935,4915513,-1,56853
3,2020-05-06,-1,04660935,4915514,-1,56853
4,2020-05-06,-1,04660935,4915516,-1,56853


In [104]:
hosp_cust_query = \
    """
    SELECT DISTINCT
    C.CUSTOMER_KEY, 
    C.CUSTOMER_NBR,
    C.CUSTOMER_ISELL_LOGIN,
    C.CUSTOMER_NAME,
    C.CUSTOMER_TYPE_DESC,
    C.CUSTOMER_TYPE,
    C.CUSTOMER_CATEGORY_DESC
    FROM 
    AMADM.dim_customer c 
    WHERE CUSTOMER_CATEGORY_DESC='Hospital'
    """

In [105]:
HOSP_CUST = pd.read_sql(con=AMADM, sql=hosp_cust_query)
HOSP_CUST.head()

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,1,000000105762,35HP10545,Nassau County Medical Center,Eprofile Webstore Cusomer,Eprofile,Hospital
1,17,000001051645,35HP02040,United Memorial Medical Center,Institution,Instit,Hospital
2,22,000001117945,48HP60190,West Texas Va Health Care System,Institution,Instit,Hospital
3,25,000001246802,15HP74233,Madison Memorial Hospital,Eprofile Webstore Cusomer,Eprofile,Hospital
4,28,000001401812,33HP00365,Capital Health System,Eprofile Webstore Cusomer,Eprofile,Hospital


In [37]:
hosp_may = pd.merge(HOSP_CUST, may_cred, on='CUSTOMER_KEY')

In [106]:
HOSP_CUST.CUSTOMER_KEY = HOSP_CUST.CUSTOMER_KEY.astype(str)

In [25]:
MAY_PHYS.dtypes

FULL_DT                     object
ME                          object
ORDER_NBR                   object
ORDER_PRODUCT_ID             int64
ORDER_PHYSICIAN_HIST_KEY     int64
CUSTOMER_KEY                object
dtype: object

In [88]:
may_cred = pd.merge(MAY_PHYS, ppd, on='ME')[['FULL_DT','ME','FIRST_NAME','MIDDLE_NAME', 'LAST_NAME','CUSTOMER_KEY','ORDER_PRODUCT_ID']]

In [30]:
ppd.head()

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
0,102000013,A,1,2.0,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102000021,A,1,3.0,MERI K MARTIN MD,MARTIN,MERI,KING,NaN,NaN,...,NC,27617,7408.0,R077,NaN,NaN,NaN,NaN,NaN,NaN
2,102000030,A,1,1.0,MARK T JACKSON MD,JACKSON,MARK,THOMAS,NaN,NaN,...,NC,28805,1262.0,C082,NaN,NaN,NaN,NaN,NaN,NaN
3,102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,AZ,85375,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN
4,102000056,A,1,2.0,RAJIV B GALA MD,GALA,RAJIV,BABULAL,NaN,NaN,...,LA,70115,6949.0,C003,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
ppd['ME'] = fix_me(list(ppd['ME']))

In [41]:
hosp_may[['CUSTOMER_NAME','CUSTOMER_NBR','CUSTOMER_KEY','FULL_DT','ME','FIRST_NAME','MIDDLE_NAME','LAST_NAME']].to_csv('../../Data/Credentialling/Credentialing_Order_Affiliation_Product_2020_05.csv')

In [75]:
MAY_STUFF = hosp_may[['CUSTOMER_NAME','CUSTOMER_NBR','CUSTOMER_KEY','FULL_DT','ME','FIRST_NAME','MIDDLE_NAME','LAST_NAME']]

In [43]:
HOSP_CUST.to_csv('../../Data/Credentialling/Hospital_Customers.csv')

In [44]:
HOSP_CUST

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,1,000000105762,35HP10545,Nassau County Medical Center,Eprofile Webstore Cusomer,Eprofile,Hospital
1,17,000001051645,35HP02040,United Memorial Medical Center,Institution,Instit,Hospital
2,22,000001117945,48HP60190,West Texas Va Health Care System,Institution,Instit,Hospital
3,25,000001246802,15HP74233,Madison Memorial Hospital,Eprofile Webstore Cusomer,Eprofile,Hospital
4,28,000001401812,33HP00365,Capital Health System,Eprofile Webstore Cusomer,Eprofile,Hospital
...,...,...,...,...,...,...,...
8980,136911,000002083613,05HP00497,Emanate Health,Eprofile Webstore Customer,Eprofile,Hospital
8981,136916,000003863708,None,Rosebud Health Care Center,Eprofile Webstore Customer,Eprofile,Hospital
8982,136941,000002080895,46HR20097,Avera Health,Eprofile Webstore Customer,Eprofile,Hospital
8983,136943,000002083432,03HP09203,Southern Arizona Va Healthcare Sys,Eprofile Webstore Customer,Eprofile,Hospital


In [9]:
npi_hosp = pd.read_csv('../../Data/NPI/NPI_Hospitals_20-01-12.csv')
amc_hosp = pd.read_csv('../../Data/NPI/AMC_Hospital_Dataset_20-02-18.csv')
len(npi_hosp)

25249

In [58]:
# HOSP_CUST['CUSTOMER_NAME'] = HOSP_CUST['CUSTOMER_NAME'].str.upper()


on_name = pd.merge(HOSP_CUST, npi_hosp, left_on = 'CUSTOMER_NAME', right_on = 'NAME')
on_other_name = pd.merge(HOSP_CUST, npi_hosp, left_on = 'CUSTOMER_NAME', right_on = 'OTHER_NAME')


In [73]:
HOSP_CUST[HOSP_CUST.CUSTOMER_NBR.isin(list(npi_match.CUSTOMER_NBR))==False].to_csv('../../Data/Credentialling/UNMATCHED.csv', index=False)

In [66]:
npi_match = all_match.sort_values('LAST_UPDATE_DATE').drop_duplicates('CUSTOMER_NBR', keep ='last')

In [63]:
# all_match = pd.concat([on_name, on_other_name])
all_match['LAST_UPDATE_DATE'] = pd.to_datetime(all_match['LAST_UPDATE_DATE'])

In [52]:
HOSP_CUST = HOSP_CUST.drop_duplicates('CUSTOMER_NBR')

In [51]:
hosp_may[['CUSTOMER_NAME','CUSTOMER_NBR','CUSTOMER_KEY','FULL_DT','ME','FIRST_NAME','MIDDLE_NAME','LAST_NAME']].drop_duplicates()

,CUSTOMER_NAME,CUSTOMER_NBR,CUSTOMER_KEY,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME
0,River Region Health System,000002099306,51732,2020-05-11,03509131154,JONATHAN,HANS,CHEN
1,River Region Health System,000002099306,51732,2020-05-13,69018050030,OLUWAROTIMI,ADEYEMI,ADEPOJU
2,River Region Health System,000002099306,51732,2020-05-27,02106150535,RYAN,WILLIAM,BROWN
3,River Region Health System,000002099306,51732,2020-05-18,49570110097,THULASI,RAM,GUDI
4,River Region Health System,000002099306,51732,2020-05-07,02407871262,JOAN,HAZEL,SUTCLIFFE
...,...,...,...,...,...,...,...,...
22118,Kingman Regional Medical Center,000002080332,136726,2020-05-30,02312980374,WILLIAM,NaN,HALLIER
22119,George Washington Medical Faculty Associates,000002080801,136743,2020-05-29,02101880187,KIM,MARIA,CALLWOOD
22121,George Washington Medical Faculty Associates,000002080801,136743,2020-05-28,02512881192,YVONNE,RENE,REID
22123,George Washington Medical Faculty Associates,000002080801,136743,2020-05-28,49607720150,VIMLA,NaN,BHOOSHAN


In [101]:
check_12 = pd.merge(npi_match, MAY_STUFF, on = 'CUSTOMER_NBR')

In [104]:
check_12.to_csv('../../Data/Credentialling/Credentialing_Order_Affiliation_Product.csv', index=False)

In [79]:
len(MAY_STUFF)

22127

In [98]:
states = ppd[['ME','POLO_STATE']]

In [82]:
len(MAY_PHYS)

48932

In [84]:
len(hosp_may)

22127

In [86]:
hosp_may.groupby('CUSTOMER_NAME').count()

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME
CUSTOMER_NAME,,,,,,,,,,,
Abbeville General Hospital,13,13,13,13,13,13,13,13,13,12,13
Abington Memorial Hospital,28,28,28,28,28,28,28,28,28,21,28
Abraham Lincoln Memorial Hospital,2,2,2,2,2,2,2,2,2,1,2
Accel Rehabilitation Hospital Of Plano,3,3,0,3,3,3,3,3,3,1,3
Acmh Hospital,4,4,4,4,4,4,4,4,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...
Yale New Haven Hospital,2,2,2,2,2,2,2,2,2,1,2
Yavapai Regional Medical Center,2,2,2,2,2,2,2,2,2,1,2
Yuma Regional Medical Center,9,9,9,9,9,9,9,9,9,5,9


In [89]:
UNQ_HOSP = hosp_may.drop_duplicates('CUSTOMER_NAME')

In [95]:
XX = UNQ_HOSP[UNQ_HOSP.CUSTOMER_NBR.isin(list(npi_match.CUSTOMER_NBR))==False]

In [100]:
pd.merge(XX, states, on='ME').to_csv('../../Data/Credentialling/may_unmatched.csv', index=False)

In [105]:
len(HOSP_CUST)

4133

In [106]:
len(npi_match)

2173

In [107]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [11]:
HOSP_CUST = HOSP_CUST.fillna('None')
customers = [x.upper() for x in list(HOSP_CUST['CUSTOMER_NAME'].unique())]

In [14]:
HOSP_CUST['CUSTOMER_NAME'] = [x.upper() for x in list(HOSP_CUST['CUSTOMER_NAME'])]

In [26]:
npi_hosp['LAST_UPDATE_DATE'] = pd.to_datetime(npi_hosp['LAST_UPDATE_DATE'])

In [31]:
npi_hosp = npi_hosp.sort_values('LAST_UPDATE_DATE', ascending=False)
npi_hosp_bop = npi_hosp.drop_duplicates(['NAME'], keep='first')

In [33]:
npi_hospitals = list(npi_hosp_bop['NAME'])

In [39]:
dict_list = []
count = 0
for customer in customers:
    if customer in npi_hospitals:
        npi_name = customer
        high_score = 100
    else:
        high_score = 0
        for hospital in npi_hospitals:
            score = fuzz.ratio(hospital, customer)
            if score>90:
                high_score = score
                npi_name = hospital
                break
            elif score>high_score:
                high_score = score
                npi_name = hospital
    new_dict = {
        'CUSTOMER_NAME': customer,
        'NAME': npi_name,
        'SCORE': high_score
        }
    dict_list.append(new_dict)
    count += 1
    print(f'{customer} is {npi_name}, {high_score}')
    print(f'{count} customers matched')
    print('----')
    print('')

NASSAU COUNTY MEDICAL CENTER is ASHTABULA COUNTY MEDICAL CENTER, 88
1 customers matched
----

UNITED MEMORIAL MEDICAL CENTER is UNITED MEMORIAL MEDICAL CENTER, 100
2 customers matched
----

WEST TEXAS VA HEALTH CARE SYSTEM is WEST TEXAS VA HEALTH CARE SYSTEM, 100
3 customers matched
----

MADISON MEMORIAL HOSPITAL is MADISON CO MEMORIAL HOSPITAL, 94
4 customers matched
----

CAPITAL HEALTH SYSTEM is CAPITAL HEALTH SYSTEM, 100
5 customers matched
----

NIAGARA FALLS MEMORIAL MEDICAL CENTER is NIAGARA FALLS MEMORIAL MEDICAL CENTER, 100
6 customers matched
----

MINNEAPOLIS VA MEDICAL CENTER is MINNEAPOLIS VA MEDICAL CENTER, 100
7 customers matched
----

KINGS COUNTY HOSPITAL CENTER is KINGS COUNTY HOSPITAL CENTER, 100
8 customers matched
----

SOUTH SHORE HOSPITAL is SOUTH SHORE HOSPITAL INC., 89
9 customers matched
----

ORLANDO HEALTH CENTRAL INC is ORLANDO HEALTH CENTRAL, 92
10 customers matched
----

SOUTH CENTRAL REGIONAL MEDICAL CENTER is SOUTH CENTRAL REGIONAL MEDICAL CENTER, 100


CENTRA HEALTH is CENTURA HEALTH, 96
93 customers matched
----

CENTRAL WASHINGTON HOSPITAL is THE WASHINGTON HOSPITAL, 84
94 customers matched
----

UNIVERSITY OF COLORADO HOSPITAL is UNIVERSITY OF COLORADO HOSPITAL, 100
95 customers matched
----

WEST VIRGINIA UNIVERSITY HOSPITALS is WEST VIRGINIA UNIVERSITY HOSPITALS, INC, 93
96 customers matched
----

TEMPLE UNIVERSITY HOSPITAL is TEMPLE UNIVERSITY HOSPITAL, 100
97 customers matched
----

ALLEGHENY HEALTH NETWORK is ALLEGHENY HEALTH NETWORK, 100
98 customers matched
----

TRIHEALTH is TRIHEALTH, 100
99 customers matched
----

REGIONAL MED CTR OF ORANGEBURG & CALHOUN is THE REGIONAL MEDICAL CENTER OF ORANGEBURG AND CALHOUN COUNTIES, 76
100 customers matched
----

PROVIDENCE HOSPITAL is PROVIDENCE HOSPITAL, 100
101 customers matched
----

ROCKDALE MEDICAL CENTER is ROCKDALE MEDICAL CENTER INC., 90
102 customers matched
----

PARKLAND HEALTH & HOSPITAL SYSTEM is PARKLAND HEALTH AND HOSPITAL SYSTEM, 94
103 customers matched
----

ONSLOW

ST LUKES HEALTH SYSTEM is ST. LUKES HEALTH SYSTEM, 98
185 customers matched
----

BEACON HEALTH STRATEGIES, LLC is BEACON HEALTH SERVICES INC, 76
186 customers matched
----

SCOTT AND WHITE is SCOTT AND WHITE, 100
187 customers matched
----

PEACE HEALTH CVO is PEACEHEALTH, 81
188 customers matched
----

SOUTHERN ILLINOIS HOSPITAL SERVICES is SOUTHERN ILLINOIS HOSPITAL SERVICES, 100
189 customers matched
----

MAIMONIDES MEDICAL CENTER is MAIMONIDES MEDICAL CENTER, 100
190 customers matched
----

MAUI MEMORIAL MEDICAL CENTER is MAUI MEMORIAL MEDICAL CENTER, 100
191 customers matched
----

MARGARET MARY HEALTH is MERCY HEALTH, 69
192 customers matched
----

JOINT TOWNSHIP DISTRICT MEMORIAL HOSP. is JOINT TOWNSHIP DISTRICT MEMORIAL HOSPITAL, 94
193 customers matched
----

KAISER PERMANENTE - REGION SOUTH is KAISER PERMANENTE ORANGE COUNTY, 76
194 customers matched
----

KENNEDY HEALTH SYSTEM is KENNEDY HEALTH SYSTEM, 100
195 customers matched
----

LAKELAND REGIONAL MEDICAL CENTER is LAK

HCA - NASHVILLE CREDENTIALING PROCESSING CENTER is ARKANSAS VALLEY REGIONAL MEDICAL CENTER, 56
277 customers matched
----

FOREST PARK MEDICAL CENTER is FOREST PARK MEDICAL CENTER LLC, 93
278 customers matched
----

TEAMHEALTH-NORTHWEST is TEAMHEALTH, 67
279 customers matched
----

HOSPITAL METROPOLITANO DR. PILA is HOSPITAL MEROPOLITANO DR. TITO MATTEI, 82
280 customers matched
----

HOAG HOSPITAL is HOAG HOSPITAL, 100
281 customers matched
----

OUR LADY OF THE ANGELS HOSPITAL is OUR LADY OF THE ANGELS HOSPITAL, INC., 91
282 customers matched
----

OUR LADY OF LOURDES REGIONAL MED.CTR.INC is OUR LADY OF LOURDES REGIONAL MEDICAL CENTER INC, 87
283 customers matched
----

NAVAL MEDICAL CENTER is VA MEDICAL CENTER, 92
284 customers matched
----

SEATTLE CHILDREN'S HOSPITAL is SEATTLE CHILDREN'S HOSPITAL, 100
285 customers matched
----

BEAUREGARD MEMORIAL HOSPITAL is SPEARE MEMORIAL HOSPITAL, 85
286 customers matched
----

WEATHERFORD REGIONAL MEDICAL CENTER is GREATER REGIONAL MEDICAL 

FOREST PARK FRISCO is FORT HARRISON VA, 65
366 customers matched
----

ERNEST HEALTH, INC. is CENTRA HEALTH, INC., 84
367 customers matched
----

DREXEL UNIVERSITY is DUKE UNIVERSITY, 81
368 customers matched
----

UNC HCS CENTRALIZED CREDENTIALING OFFICE is UC DAVIS UNIV DENTAL ASSOCIATES, 51
369 customers matched
----

DEACONESS HEALTH PLANS is DEACONESS HEALTH SYSTEM LLC, 73
370 customers matched
----

PROVIDENCE SACRED HEART MEDICAL CENTER is PROVIDENCE SACRED HEART MEDICAL CENTER, 100
371 customers matched
----

COLUSA COMMUNITY HOSPITAL is COLUMBUS COMMUNITY HOSPITAL, 92
372 customers matched
----

THE UNIVERSITY OF VERMONT HEALTH NETWORK - CHAMPLAIN VALLEY PHYSICIANS HOSPITAL is UNITY POINT HEALTH, BLANK CHILDRENS HOSPITAL, 59
373 customers matched
----

WILLIAMSON MEDICAL CENTER is ROGER WILLIAMS MEDICAL CENTER, 85
374 customers matched
----

ROBT WOOD JOHNSON U HSP AT HAMILTON is ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL AT HAMILTON, 81
375 customers matched
----

PHELPS COUNTY 

...

BOSTON MOUNTAIN RURAL HEALTH CENTER is ALTON MENTAL HEALTH CENTER, 75
3832 customers matched
----

GIBSON GENERAL HOSPITAL is GIBSON GENERAL HOSPITAL, 100
3833 customers matched
----

ASTRIA REGIONAL MEDICAL CENTER is PARIS REGIONAL MEDICAL CENTER, 92
3834 customers matched
----

WEST PALM BEACH VA MEDICAL CENTER is WEST PALM BEACH VA MEDICAL CENTER, 100
3835 customers matched
----

CONNECTICUT MENTAL HEALTH CENTER is CONNECTICUT MENTAL HEALTH CENTER, 100
3836 customers matched
----

VAN DIEST MEDICAL CENTER is VANDERBILT MEDICAL CENTER, 86
3837 customers matched
----

NORTHWESTERN MEMORIAL HOSPITAL is NORTHWESTERN MEMORIAL HOSPITAL, 100
3838 customers matched
----

MERCY HEALTH CVO is MERCY HEALTH, 86
3839 customers matched
----

ACADIA HOSPITAL is ACADIA HOSPITAL, 100
3840 customers matched
----

BEAUMONT HEALTH DEARBORN/MED EDUCATION is SUMMIT HEALTHCARE ASSOCIATION, 60
3841 customers matched
----

CUMBERLAND SURGICAL HOSPITAL is CUMBERLAND MEMORIAL HOSPITAL, 86
3842 customers match

NORTH MISSISSIPPI MEDICAL CENTER GILMORE AMORY is NORTH MISSISSIPPI MEDICAL CENTER, 82
3920 customers matched
----

MEMORIAL REGIONAL HEALTH SERVICES is RANGE REGIONAL HEALTH SERVICES, 86
3921 customers matched
----

SANTA CRUZ VALLEY REGIONAL HOSPITAL is CENTRAL REGIONAL HOSPITAL, 77
3922 customers matched
----

WINNEBAGO HEALTH CARE SYSTEMS is VA HEALTH CARE SYSTEMS, 82
3923 customers matched
----

WEST TENNESSEE HEALTHCARE DYERSBURG HOSPITAL is SCOTTSDALE HEALTHCARE HOSPITALS, 67
3924 customers matched
----

CALVARY HOSPITAL is CALVARY HOSPITAL, 100
3925 customers matched
----

ADVENTHEALTH DAYTONA BEACH is ADVENTIST HEALTH TULARE, 65
3926 customers matched
----

CORAL SHORES BEHAVIORAL HEALTH is CORAL SHORES BEHAVIORAL HEALTH, LLC, 92
3927 customers matched
----

CHI ST. VINCENT HOT SPRINGS REHABILITATION HOSPITAL is HOT SPRINGS REHABILITATION CENTER HOSPITAL, 75
3928 customers matched
----

ST. JOSEPH HEALTH NORTH CALIFORNIA REGIONAL LLC is ST. JOSEPH HEALTH NORTHERN CALIFORNIA, L

DEACTIVATED - CONEY ISLAND HOSPITAL is CONEY ISLAND HOSPITAL, 75
4002 customers matched
----

DEACTIVATED - METROPOLITAN HOSPITAL is METROPOLITAN HOSPITAL, 75
4003 customers matched
----

DEACTIVATED - HARLEM HOSPITAL CENTER is HARLEM HOSPITAL CENTER, 76
4004 customers matched
----

DEACTIVATED - JACOBI MEDICAL CENTER AND/OR NORTH CENTRAL BRONX HOSPITAL is EAST TEXAS MEDICAL CENTER REHABILITATION HOSPITAL, 62
4005 customers matched
----

DEACTIVATED - BELLEVUE HOSPITAL CENTER is BELLEVUE HOSPITAL CENTER, 77
4006 customers matched
----

ADVENTHEALTH OTTAWA is ADVENTIST HEALTH TULARE, 71
4007 customers matched
----

FIRSTHEALTH OF THE CAROLINAS is FIRSTHEALTH OF THE CAROLINAS, 100
4008 customers matched
----

CLEVELAND CLINIC MARTIN HEALTH is CLEVELAND CLINIC AVON HOSPITAL, 77
4009 customers matched
----

METROHEALTH MEDICAL CENTER is METROHEALTH MEDICAL CENTER, 100
4010 customers matched
----

PROMEDICA is RTGMEDICAL, 74
4011 customers matched
----

TEXAS HEALTH HUGULEY HOSPITAL is TEXA

ENCOMPASS HEALTH REHABILITATION HOSPITAL OF NITTANY VALLEY is ENCOMPASS HEALTH REHABILITATION HOSPITAL OF NITTANY VALLEY, INC., 95
4067 customers matched
----

ENCOMPASS HEALTH REHABILITATION HOSPITAL OF SEWICKLEY is ENCOMPASS HEALTH REHABILITATION HOSPITAL OF SEWICKLEY, LLC, 95
4068 customers matched
----

ENCOMPASS HEALTH REHABILITATION HOSPITAL OF BRAINTREE is ENCOMPASS HEALTH REHABILITATION HOSPITAL OF BRAINTREE, LLC, 95
4069 customers matched
----

ENCOMPASS HEALTH REHABILITATION HOSPITAL OF LARGO is ENCOMPASS HEALTH REHABILITATION HOSPITAL OF LARGO, LLC, 95
4070 customers matched
----

ENCOMPASS HEALTH REHABILITATION HOSPITAL OF ABILENE is ENCOMPASS HEALTH REHABILITATION HOSPITAL OF UTAH LLC, 89
4071 customers matched
----

YUMA REHABILITATION HOSPITAL is MAGEE REHABILITATION HOSPITAL, 91
4072 customers matched
----

ENCOMPASS HEALTH REHABILITATION HOSPITAL OF DAYTON is ENCOMPASS HEALTH REHABILITATION HOSPITAL OF DAYTON, LLC, 95
4073 customers matched
----

ENCOMPASS HEALTH REHAB

MODERN MEDICINE SURGICAL CENTER is FRESNO SURGICAL CENTER, 72
4135 customers matched
----

SSM HEALTH DEPAUL HOSPITAL is MISSION HEALTH AND HOSPITALS, 74
4136 customers matched
----

STONY BROOK EASTERN LONG ISLAND HOSPITAL is STONY BROOK UNIVERSITY HOSPITAL, 68
4137 customers matched
----

ADVENTIST HEALTHCARE WHITE OAK MEDICAL CENTER is ADVENTIST HEALTHCARE FORT WASHINGTON MEDICAL CENTER INC, 82
4138 customers matched
----

COMPREHENSIVE CARE AND RESEARCH CENTER, TULSA is COMPREHENSIVE CANCER CENTERS OF NEVADA, 65
4139 customers matched
----

SAND HILLS CENTER is SANFORD MEDICAL CENTER, 72
4140 customers matched
----

SURGICAL HOSPITAL OF OKLAHOMA is SURGICAL HOSPITAL OF OKLAHOMA LLC, 94
4141 customers matched
----

BRUSHY CREEK HOSPITAL is BRUSHY CREEK FAMILY HOSPITAL, LLC, 78
4142 customers matched
----

KAHI MOHALA is KAHI MOHALA, 100
4143 customers matched
----

PRESBYTERIAN HOSPITAL ASC, LLC is PRESBYTERIAN HOSPITAL OF DALLAS, 82
4144 customers matched
----

WEST SUBURBAN MEDICA

In [36]:
len(npi_hospitals)

15289

In [43]:
# reference = pd.DataFrame(dict_list)
pd.merge(reference, npi_hosp_bop, on='NAME').to_csv('../../Data/Credentialling/Reference_Match.csv', index=False)

In [41]:
pd.DataFrame(dict_list).to_csv('../../Data/Credentialling/Reference_Match.csv', index=False)

,CUSTOMER_NAME,NAME,SCORE
0,NASSAU COUNTY MEDICAL CENTER,ASHTABULA COUNTY MEDICAL CENTER,88
1,UNITED MEMORIAL MEDICAL CENTER,UNITED MEMORIAL MEDICAL CENTER,100
2,WEST TEXAS VA HEALTH CARE SYSTEM,WEST TEXAS VA HEALTH CARE SYSTEM,100
3,MADISON MEMORIAL HOSPITAL,MADISON CO MEMORIAL HOSPITAL,94
4,CAPITAL HEALTH SYSTEM,CAPITAL HEALTH SYSTEM,100
...,...,...,...
4216,EMANATE HEALTH,MAINEHEALTH,80
4217,ROSEBUD HEALTH CARE CENTER,HOPI HEALTH CARE CENTER,82
4218,AVERA HEALTH,ARIA HEALTH,87
4219,HARNEY DISTRICT HOSPITAL,HARNEY DISTRICT HOSPITAL,100


In [37]:
customers

['NASSAU COUNTY MEDICAL CENTER',
 'UNITED MEMORIAL MEDICAL CENTER',
 'WEST TEXAS VA HEALTH CARE SYSTEM',
 'MADISON MEMORIAL HOSPITAL',
 'CAPITAL HEALTH SYSTEM',
 'NIAGARA FALLS MEMORIAL MEDICAL CENTER',
 'MINNEAPOLIS VA MEDICAL CENTER',
 'KINGS COUNTY HOSPITAL CENTER',
 'SOUTH SHORE HOSPITAL',
 'ORLANDO HEALTH CENTRAL INC',
 'SOUTH CENTRAL REGIONAL MEDICAL CENTER',
 'SONORA REGIONAL MEDICAL',
 'PAMPA REGIONAL MEDICAL CENTER',
 'BAPTIST HEALTH RICHMOND',
 'MIDLAND MEMORIAL HOSPITAL',
 "CHILDREN'S HOSPITAL COLORADO",
 'ALEXIAN BROTHERS HEALTH SYSTEM',
 'SIDRA MEDICAL & RESEARCH CENTER',
 'CHARLESTON AREA MEDICAL CENTER INC.',
 'DOYLESTOWN HOSPITAL',
 'MEADOWLANDS HOSPITAL MED CTR',
 'HONOR HEALTH',
 'DIGNITY HEALTH',
 'EDWARD HOSPITAL',
 'KIRBY MEDICAL CENTER',
 'UCI MEDICAL CENTER',
 'COVENANT HEALTH SYSTEM',
 'VERMONT MANAGED CARE',
 'CAROLINAS MEDICAL CENTER',
 'ALASKA NATIVE MEDICAL CENTER',
 'ALTRU HEALTH SYSTEM',
 'BATON ROUGE GENERAL MEDICAL CENTER',
 'BRONSON BATTLE CREEK',
 'BAP

In [47]:
manual = pd.read_csv('../../Data/Credentialling/may_manual.csv')

In [48]:
manual = manual.fillna('None')

In [51]:
len(manual.CUSTOMER_KEY.unique())

885

In [61]:
DICT_LIST = []
for row in manual.itertuples():
    if row.city=='None' and row.address!='None':
        
        print(row.address)
        add_list = row.address.split(', ')
        address = add_list[0]
        print(address)
        try:
            city = add_list[1]
            state = add_list[2].split(' ')[0]
            zipcode = add_list[2].split(' ')[1]
        except:
            city = 'None'
            state = 'None'
            zipcode = 'None'
    else:
        address = row.address
        city = row.city
        state = row.state
        zipcode = row.zipcode
    new_dict = {
        'Address': address,
        'City': city,
        'State': state,
        'Zipcode': zipcode,
        'CUSTOMER_KEY':row.CUSTOMER_KEY
    }
    DICT_LIST.append(new_dict)

 2601 E Roosevelt St, Phoenix, AZ 85008
 2601 E Roosevelt St
 2893 Sturgis Rd, Rock Hill, SC 29730
 2893 Sturgis Rd
1 Riverway Suite 700, Houston, TX 77056
1 Riverway Suite 700
1 Tampa General Cir, Tampa, FL 33606
1 Tampa General Cir
100 Hospital Rd, Prince Frederick, MD 20678
100 Hospital Rd, Prince Frederick, MD 20678
100 Medical Center Dr, Princeton, KY 42445
100 Medical Center Dr
1000 36th St Vero Beach, FL 32960 United States
1000 36th St Vero Beach
1000 Medical Center Dr, Hardeeville, SC 29927
1000 Medical Center Dr
10000 West Colonial Drive Ocoee, FL 34761 
10000 West Colonial Drive Ocoee
101 Dudley St, Providence, RI 02905
101 Dudley St
101 E 9th St, Pana, IL 62557
101 E 9th St
10101 Park Rowe Cir # 500, Baton Rouge, LA 70810
10101 Park Rowe Cir # 500
10105 Park Rowe Cir, Baton Rouge, LA 70810
10105 Park Rowe Cir
1011 N Galloway Ave, Mesquite, TX 75149
1011 N Galloway Ave
10333 El Camino Real, Atascadero, CA 93422
10333 El Camino Real
105 Highway 80 East, Demopolis, AL 36732
10

In [62]:
ref = pd.read_csv('../../Data/Credentialling/Reference_Match.csv')

In [63]:
ref

,CUSTOMER_NAME,NAME,SCORE,NPI,Taxonomies,OTHER_NAME,MAILING_ADDRESS_1,MAILING_ADDRESS_2,MAILING_CITY,MAILING_STATE,MAIING_ZIP,MAILING_PHONE,LOCATION_ADDRESS_1,LOCATION_ADDRESS_2,LOCATION_CITY,LOCATION_STATE,LOCATION_ZIP,LOCATION_PHONE,LAST_UPDATE_DATE
0,633D MEDICAL GROUP/SGHC,633D MEDICAL GROUP,88.0,1.609860e+09,"['261QM1100X', '261QM1101X', '2865M2000X', '33...",1ST MEDICAL GROUP,77 NEALY AVE,None,LANGLEY AFB,VA,236652040,7577646807,77 NEALY AVE,None,LANGLEY AFB,VA,236652040.0,7577646807,3/3/2014
1,A.O. FOX MEMORIAL HOSPITAL,A.O FOX MEMORIAL HOSPITAL,98.0,1.104029e+09,['282N00000X'],None,2 ELLIOTTS ALY,None,VALLEY COTTAGE,NY,109892306,6462395675,1 NORTON AVE,None,ONEONTA,NY,138202629.0,6074315243,7/8/2007
2,ABBEVILLE COUNTY MEMORIAL HOSPITAL,ABBEVILLE COUNTY MEMORIAL HOSPITAL,100.0,1.063449e+09,"['282NC0060X', '3336C0003X']",ABBEVILLE AREA MEDICAL CENTER,PO BOX 887,None,ABBEVILLE,SC,296200887,8643363249,420 THOMSON CIR,None,ABBEVILLE,SC,29620.0,8643665011,8/23/2018
3,ABBEVILLE GENERAL HOSPITAL,ABBEVILLE GENERAL HOSPITAL,100.0,1.255300e+09,['282NR1301X'],None,118 N HOSPITAL DR,None,ABBEVILLE,LA,705104039,3378935466,118 N HOSPITAL DR,None,ABBEVILLE,LA,705104039.0,3378935466,4/20/2008
4,ABILENE BEHAVIORAL HEALTH,"ABILENE BEHAVIORAL HEALTH, LLC",91.0,1.740332e+09,['283Q00000X'],"ACADIA ABILENE, LLC",830 CRESENT CENTRE DRIVE,SUITE 610,FRANKLIN,TN,37067,6158616000,4225 WOODS PLACE,None,ABILENE,TX,79602.0,3256986600,7/30/2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,SAGE REHABILITATION HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4217,UNION COUNTY HOSPITAL DISTRICT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4218,MARTIN GENERAL HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4219,DVA MEDICAL CENTER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
ZZ = pd.merge(ref, MAN_ADDS, on='CUSTOMER_NAME', how='left')

In [66]:
adds = pd.DataFrame(DICT_LIST)

In [68]:
MAN_ADDS = pd.merge(adds, manual, on='CUSTOMER_KEY')

In [74]:
ZZ=ZZ.fillna('None')
ZZ[(ZZ.npi=='None')&(ZZ.NPI=='None')&(ZZ.address=='None')]

,CUSTOMER_NAME,NAME,SCORE,NPI,Taxonomies,OTHER_NAME,MAILING_ADDRESS_1,MAILING_ADDRESS_2,MAILING_CITY,MAILING_STATE,...,MIDDLE_NAME,LAST_NAME,POLO_STATE,address,city,state,zipcode,other name,npi,Google state
2455,DEACTIVATED - SEA VIEW HOSP REHAB CTR & HOME,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2456,ARMY RESERVE CLINICAL CREDENTIALING AFFAIRS,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2457,UNC HCS CENTRALIZED CREDENTIALING OFFICE,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2458,NHRMC/PENDER,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2459,DEACTIVATED - HENRY J. CARTER,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,SAGE REHABILITATION HOSPITAL,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4217,UNION COUNTY HOSPITAL DISTRICT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4218,MARTIN GENERAL HOSPITAL,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4219,DVA MEDICAL CENTER,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [75]:
ZZ

,CUSTOMER_NAME,NAME,SCORE,NPI,Taxonomies,OTHER_NAME,MAILING_ADDRESS_1,MAILING_ADDRESS_2,MAILING_CITY,MAILING_STATE,...,MIDDLE_NAME,LAST_NAME,POLO_STATE,address,city,state,zipcode,other name,npi,Google state
0,633D MEDICAL GROUP/SGHC,633D MEDICAL GROUP,88,1.60986e+09,"['261QM1100X', '261QM1101X', '2865M2000X', '33...",1ST MEDICAL GROUP,77 NEALY AVE,None,LANGLEY AFB,VA,...,None,None,None,None,None,None,None,None,None,None
1,A.O. FOX MEMORIAL HOSPITAL,A.O FOX MEMORIAL HOSPITAL,98,1.10403e+09,['282N00000X'],None,2 ELLIOTTS ALY,None,VALLEY COTTAGE,NY,...,None,None,None,None,None,None,None,None,None,None
2,ABBEVILLE COUNTY MEMORIAL HOSPITAL,ABBEVILLE COUNTY MEMORIAL HOSPITAL,100,1.06345e+09,"['282NC0060X', '3336C0003X']",ABBEVILLE AREA MEDICAL CENTER,PO BOX 887,None,ABBEVILLE,SC,...,None,None,None,None,None,None,None,None,None,None
3,ABBEVILLE GENERAL HOSPITAL,ABBEVILLE GENERAL HOSPITAL,100,1.2553e+09,['282NR1301X'],None,118 N HOSPITAL DR,None,ABBEVILLE,LA,...,None,None,None,None,None,None,None,None,None,None
4,ABILENE BEHAVIORAL HEALTH,"ABILENE BEHAVIORAL HEALTH, LLC",91,1.74033e+09,['283Q00000X'],"ACADIA ABILENE, LLC",830 CRESENT CENTRE DRIVE,SUITE 610,FRANKLIN,TN,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4216,SAGE REHABILITATION HOSPITAL,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4217,UNION COUNTY HOSPITAL DISTRICT,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4218,MARTIN GENERAL HOSPITAL,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4219,DVA MEDICAL CENTER,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [90]:
may_cred.dtypes

FULL_DT             object
ME                  object
FIRST_NAME          object
MIDDLE_NAME         object
LAST_NAME           object
CUSTOMER_KEY        object
ORDER_PRODUCT_ID     int64
dtype: object

In [89]:
pd.merge(may_cred, ZZ, on='CUSTOMER_KEY')

,FULL_DT_x,ME_x,FIRST_NAME_x,MIDDLE_NAME_x,LAST_NAME_x,CUSTOMER_KEY,ORDER_PRODUCT_ID,CUSTOMER_NAME,NAME,SCORE,...,MIDDLE_NAME_y,LAST_NAME_y,POLO_STATE,address,city,state,zipcode,other name,npi,Google state


In [95]:
MAN_ADDS

,Address,City,State,Zipcode,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,...,MIDDLE_NAME,LAST_NAME,POLO_STATE,address,city,state,zipcode,other name,npi,Google state
0,2601 E Roosevelt St,Phoenix,AZ,85008,131103,2188824,03HP09532,Valleywise Health Medical Center,Eprofile Webstore Customer,Eprofile,...,MARIE,KHALSA,AZ,"2601 E Roosevelt St, Phoenix, AZ 85008",None,None,None,None,1.07358e+09,None
1,2893 Sturgis Rd,Rock Hill,SC,29730,60149,2102241,45AM80123,Catawba Service Unit,Eprofile Webstore Customer,Eprofile,...,TYRON,BELL,NC,"2893 Sturgis Rd, Rock Hill, SC 29730",None,None,None,None,None,SC
2,8101 W Sam Houston Pkwy S # Sui,"Houston, TX 77072",None,None,59997,2081195,48HP64839,Hca Shared Services Center-Houston,Eprofile Webstore Customer,Eprofile,...,SHAY,WHITE,AR,8101 W Sam Houston Pkwy S # Sui,"Houston, TX 77072",None,None,None,None,TX
3,1 Riverway Suite 700,Houston,TX,77056,60526,2083557,48HP30683,Nexus Health Systems,Eprofile Webstore Customer,Eprofile,...,GALVAN,UMNAS,CA,"1 Riverway Suite 700, Houston, TX 77056",None,None,None,None,None,TX
4,1 Tampa General Cir,Tampa,FL,33606,131621,2083242,11HP40055,Tampa General Healthcare,Eprofile Webstore Customer,Eprofile,...,JUNE,O'CONNELL,DC,"1 Tampa General Cir, Tampa, FL 33606",None,None,None,None,None,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,None,None,None,None,58587,2086739,56HP00763,"Wheaton Franciscan Hlthcare- All Saints, Inc.",Eprofile Webstore Customer,Eprofile,...,KAY,ZERFAS,None,None,None,None,None,None,None,None
881,None,None,None,None,60294,2201509,None,Willis-Knighton Health System,Eprofile Webstore Customer,Eprofile,...,MAYO,ELLIS,None,None,None,None,None,None,None,None
882,None,None,None,None,57580,2084754,48HP00956,Wilson N Jones Regional Medical Center,Eprofile Webstore Customer,Eprofile,...,None,AHMAD,None,None,None,None,None,None,None,None
883,None,None,None,None,53532,2083953,57HP63331,Wyoming Medical Center,Eprofile Webstore Customer,Eprofile,...,WEBB,KING,None,None,None,None,None,None,None,None


In [97]:
product_key = pd.read_csv('../../Data/Credentialling/product_key.csv')

In [100]:
product_key

,PRODUCT_DESC,PRODUCT_ID
0,Base Data,4915508
1,State License (Advanced),4915509
2,Bio Sketch,4915512
3,Full AMA Physician Profile,4915513
4,Reappointment AMA Physician Pr,4915514
5,Self Inquiry,4915515
6,Continuous Monitoring,4915516
7,Full AMA Physician Pro + CM,4915521
8,Reapp AMA Physician Pro + CM,4915522
9,Licensing Board,4915491


In [99]:
PROD = pd.merge(may_cred, product_key, left_on='ORDER_PRODUCT_ID', right_on='PRODUCT_ID')

In [102]:
goodkeys =[4915513, 4915514, 4915521, 4915522, 4915502]
PROD = PROD[PROD.PRODUCT_ID.isin(goodkeys)]

In [109]:
may_cust = list(PROD.drop_duplicates('CUSTOMER_KEY')['CUSTOMER_KEY'])

In [111]:
HOSP_CUST[HOSP_CUST.CUSTOMER_KEY.isin(may_cust)].

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
4276,51732,000002099306,27HP50130,River Region Health System,Eprofile Webstore Customer,Eprofile,Hospital
4278,51739,000002083534,48HP62528,Lubbock Heart & Surgical Hospital,Eprofile Webstore Customer,Eprofile,Hospital
4279,51741,000002435304,None,Oro Valley Hospital,Eprofile Webstore Customer,Eprofile,Hospital
4280,51742,000002085108,39HP07625,Creek Nation,Eprofile Webstore Customer,Eprofile,Hospital
4282,51744,000002320655,23HP90001,University Of Maryland Capital Region Health,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...
8975,136578,000002080801,54HR50121,"George Washington Univ Medical Fac Associates,...",Eprofile Webstore Customer,Eprofile,Hospital
8976,136655,000002080801,54HR50121,None,Eprofile Webstore Customer,Eprofile,Hospital
8977,136724,000002083004,33HP30241,University Hospital,Eprofile Webstore Customer,Eprofile,Hospital
8978,136726,000002080332,03HP00403,Kingman Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital


In [117]:
HOSP_CUST=HOSP_CUST.fillna('None')
HOSP_CUST['CUSTOMER_NAME']=[x.upper() for x in list(HOSP_CUST['CUSTOMER_NAME'])]

In [119]:
ref_2 = pd.merge(ref, HOSP_CUST, on='CUSTOMER_NAME')

In [113]:
manual

,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,POLO_STATE,address,city,state,zipcode,other name,npi,Google state
0,131103,2188824,03HP09532,Valleywise Health Medical Center,Eprofile Webstore Customer,Eprofile,Hospital,5/11/2020,506870640,ANN,MARIE,KHALSA,AZ,"2601 E Roosevelt St, Phoenix, AZ 85008",None,None,None,None,1.07358e+09,None
1,60149,2102241,45AM80123,Catawba Service Unit,Eprofile Webstore Customer,Eprofile,Hospital,5/29/2020,3601860081,JOSEPH,TYRON,BELL,NC,"2893 Sturgis Rd, Rock Hill, SC 29730",None,None,None,None,None,SC
2,59997,2081195,48HP64839,Hca Shared Services Center-Houston,Eprofile Webstore Customer,Eprofile,Hospital,5/29/2020,401010467,JUSTIN,SHAY,WHITE,AR,8101 W Sam Houston Pkwy S # Sui,"Houston, TX 77072",None,None,None,None,TX
3,60526,2083557,48HP30683,Nexus Health Systems,Eprofile Webstore Customer,Eprofile,Hospital,5/18/2020,519951673,GILBERT,GALVAN,UMNAS,CA,"1 Riverway Suite 700, Houston, TX 77056",None,None,None,None,None,TX
4,131621,2083242,11HP40055,Tampa General Healthcare,Eprofile Webstore Customer,Eprofile,Hospital,5/7/2020,5107990013,KAREN,JUNE,O'CONNELL,DC,"1 Tampa General Cir, Tampa, FL 33606",None,None,None,None,None,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,58587,2086739,56HP00763,"Wheaton Franciscan Hlthcare- All Saints, Inc.",Eprofile Webstore Customer,Eprofile,Hospital,5/13/2020,2507081773,DORENE,KAY,ZERFAS,None,None,None,None,None,None,None,None
881,60294,2201509,None,Willis-Knighton Health System,Eprofile Webstore Customer,Eprofile,Hospital,5/12/2020,2878880288,WILLIAM,MAYO,ELLIS,None,None,None,None,None,None,None,None
882,57580,2084754,48HP00956,Wilson N Jones Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital,5/9/2020,16002700194,NASIHA,None,AHMAD,None,None,None,None,None,None,None,None
883,53532,2083953,57HP63331,Wyoming Medical Center,Eprofile Webstore Customer,Eprofile,Hospital,5/26/2020,2604851095,MARGARET,WEBB,KING,None,None,None,None,None,None,None,None


In [123]:
ref_2 = ref_2.fillna('None')
ref_2 = ref_2[ref_2.NPI!='None']

In [126]:
ref_2[ref_2.CUSTOMER_KEY.isin(may_cust)]

,CUSTOMER_NAME,NAME,SCORE,NPI,Taxonomies,OTHER_NAME,MAILING_ADDRESS_1,MAILING_ADDRESS_2,MAILING_CITY,MAILING_STATE,...,LOCATION_STATE,LOCATION_ZIP,LOCATION_PHONE,LAST_UPDATE_DATE,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
7,ABBEVILLE GENERAL HOSPITAL,ABBEVILLE GENERAL HOSPITAL,100,1.2553e+09,['282NR1301X'],None,118 N HOSPITAL DR,None,ABBEVILLE,LA,...,LA,7.05104e+08,3378935466,4/20/2008,59647,000002084221,21HP01845,Eprofile Webstore Customer,Eprofile,Hospital
11,ABINGTON MEMORIAL HOSPITAL,ABINGTON MEMORIAL HOSPITAL,100,1.81199e+09,['282N00000X'],None,1200 OLD YORK RD,None,ABINGTON,PA,...,PA,1.90014e+08,2154812000,4/15/2010,51966,000002077717,41HP06132,Eprofile Webstore Customer,Eprofile,Hospital
13,ABRAHAM LINCOLN MEMORIAL HOSPITAL,ABRAHAM LINCOLN MEMORIAL HOSPITAL,100,1.7303e+09,"['207L00000X', '282N00000X']",None,200 STAHLHUT DR,None,LINCOLN,IL,...,IL,6.26565e+08,2177322161,6/20/2019,58591,000002084621,16HP60687,Eprofile Webstore Customer,Eprofile,Hospital
21,ACMH HOSPITAL,ACMH HOSPITAL,100,1.60994e+09,['282N00000X'],ARMSTRONG COUNTY MEMORIAL HOSPITAL,PO BOX 579,None,KITTANNING,PA,...,PA,1.62017e+08,7245438164,6/16/2008,61529,000002084667,41HP64338,Eprofile Webstore Customer,Eprofile,Hospital
31,ADENA HEALTH SYSTEM,ADENA HEALTH SYSTEM,100,1.90284e+09,['282N00000X'],ADENA REGIONAL MEDICAL CENTER,272 HOSPITAL RD,SUITE 3,CHILLICOTHE,OH,...,OH,4.56019e+08,7407797500,6/4/2014,126416,000002083580,38HR10028,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,WYCKOFF HEIGHTS MEDICAL CENTER,WYCKOFF HEIGHTS MEDICAL CENTER,100,1.70089e+09,['282N00000X'],None,374 STOCKHOLM ST,3RD FLOOR ADMINISTRATION,BROOKLYN,NY,...,NY,1.12374e+08,7189636485,4/25/2016,54610,000002083151,35HP05793,Eprofile Webstore Customer,Eprofile,Hospital
5435,WYOMING MEDICAL CENTER,WYOMING MEDICAL CENTER INC,92,1.34633e+09,['282N00000X'],None,1233 E 2ND STREET,None,CASPER,WY,...,WY,82601,3075777201,1/30/2015,53532,000002083953,57HP63331,Eprofile Webstore Customer,Eprofile,Hospital
5438,YAKIMA VALLEY MEMORIAL HOSPITAL,YAKIMA VALLEY MEMORIAL HOSPITAL,100,1.35678e+09,['284300000X'],None,3801 KERN WAY,None,YAKIMA,WA,...,WA,9.89026e+08,5095743245,6/28/2013,64370,000002083391,54HP01287,Eprofile Webstore Customer,Eprofile,Hospital
5440,YALE NEW HAVEN HOSPITAL,YALE NEW HAVEN HOSPITAL,100,1.39612e+09,['282N00000X'],None,70 CHURCH RD,None,EASTON,CT,...,CT,6.51032e+07,2037100217,8/11/2015,61090,000000533837,08HP05686,Eprofile Webstore Customer,Eprofile,Hospital


In [131]:
manual['CUSTOMER_KEY']=manual.CUSTOMER_KEY.astype(str)

In [135]:
xyz = pd.merge(ref_2, manual, on='CUSTOMER_KEY', how='outer')

In [159]:
may_results = pd.merge(PROD, xyz, on='CUSTOMER_KEY', suffixes=['_product','_reference']).drop_duplicates()

In [161]:
may_results[may_results['NPI_NUMBER']!='None'].to_excel('../../Data/Credentialling/May_Profiles.xlsx')

In [141]:
pd.merge(PROD, HOSP_CUST, on='CUSTOMER_KEY').drop_duplicates()

,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,CUSTOMER_KEY,ORDER_PRODUCT_ID,PRODUCT_DESC,PRODUCT_ID,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,2020-05-05,01720732471,DAVID,LOREN,BIXLER,52982,4915513,Full AMA Physician Profile,4915513,000002083799,05HP00484,FREMONT-RIDEOUT HEALTH GROUP,Eprofile Webstore Customer,Eprofile,Hospital
1,2020-05-29,02002140783,KRISTOFER,NaN,LANGHEINRICH,52982,4915513,Full AMA Physician Profile,4915513,000002083799,05HP00484,FREMONT-RIDEOUT HEALTH GROUP,Eprofile Webstore Customer,Eprofile,Hospital
2,2020-05-01,02608990171,STEVEN,ERIC,SIEGAL,52982,4915513,Full AMA Physician Profile,4915513,000002083799,05HP00484,FREMONT-RIDEOUT HEALTH GROUP,Eprofile Webstore Customer,Eprofile,Hospital
3,2020-05-01,49562020101,CHANDRA,SEKHAR,VELURU,52982,4915513,Full AMA Physician Profile,4915513,000002083799,05HP00484,FREMONT-RIDEOUT HEALTH GROUP,Eprofile Webstore Customer,Eprofile,Hospital
4,2020-05-08,02312101729,JONATHAN,ROBERT,GOWER,52982,4915513,Full AMA Physician Profile,4915513,000002083799,05HP00484,FREMONT-RIDEOUT HEALTH GROUP,Eprofile Webstore Customer,Eprofile,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21035,2020-05-05,04816130757,ROYCE,PAUL,GRAY,58323,4915522,Reapp AMA Physician Pro + CM,4915522,000002101397,11HP80020,EMERALD COAST BEHAVIORAL HOSPITAL,Eprofile Webstore Customer,Eprofile,Hospital
21036,2020-05-20,01875111163,BECKY,ANN,SLATER,52463,4915522,Reapp AMA Physician Pro + CM,4915522,000002084685,29HP30721,GREAT FALLS CLINIC HOSPITAL,Eprofile Webstore Customer,Eprofile,Hospital
21037,2020-05-18,03875810102,DENNIS,PATRICK,RUGGERIE,52463,4915522,Reapp AMA Physician Pro + CM,4915522,000002084685,29HP30721,GREAT FALLS CLINIC HOSPITAL,Eprofile Webstore Customer,Eprofile,Hospital
21038,2020-05-19,30803830653,JORGE,LUIS,CASTRIZ,52463,4915522,Reapp AMA Physician Pro + CM,4915522,000002084685,29HP30721,GREAT FALLS CLINIC HOSPITAL,Eprofile Webstore Customer,Eprofile,Hospital


In [139]:
len(may_cust)

3650

In [149]:
xyz = xyz.fillna('None')

In [156]:
npis=[]
for row in xyz.itertuples():
    if row.npi=='None' and row.NPI!='None':
        npi_num = row.NPI
        print('Automation')
    elif row.NPI=='None' and row.npi!='None':
        npi_num = row.npi
        print('Manual')
    elif row.NPI!=row.npi:
        npi_num = row.npi
        print('Mismatched npis')
    elif row.address!='None'and row.npi=='None':
        npi_num = 'Address'
        print('Address')
    else:
        npi_num = row.npi
    npis.append(npi_num)
    print(f'{row.CUSTOMER_NAME_x} : {npi_num}')

Automation
633D MEDICAL GROUP/SGHC : 1609859891.0
Automation
633D MEDICAL GROUP/SGHC : 1609859891.0
Automation
A.O. FOX MEMORIAL HOSPITAL : 1104029180.0
Automation
A.O. FOX MEMORIAL HOSPITAL : 1104029180.0
Automation
ABBEVILLE COUNTY MEMORIAL HOSPITAL : 1063449361.0
Automation
ABBEVILLE COUNTY MEMORIAL HOSPITAL : 1063449361.0
Automation
ABBEVILLE GENERAL HOSPITAL : 1255300414.0
Automation
ABBEVILLE GENERAL HOSPITAL : 1255300414.0
Automation
ABILENE BEHAVIORAL HEALTH : 1740331693.0
Automation
ABILENE BEHAVIORAL HEALTH : 1740331693.0
Automation
ABINGTON MEMORIAL HOSPITAL : 1811992084.0
Automation
ABINGTON MEMORIAL HOSPITAL : 1811992084.0
Automation
ABRAHAM LINCOLN MEMORIAL HOSPITAL : 1730304361.0
Automation
ABRAHAM LINCOLN MEMORIAL HOSPITAL : 1730304361.0
Automation
ACADIA HOSPITAL : 1578911830.0
Automation
ACADIA HOSPITAL : 1578911830.0
Automation
ACADIA REHABILITATION HOSPITAL, L.L.C. : 1043216583.0
Automation
ACADIA REHABILITATION HOSPITAL, L.L.C. : 1043216583.0
Automation
ACCESS HOSP

COQUILLE VALLEY HOSPITAL : 1730223967.0
Automation
COQUILLE VALLEY HOSPITAL : 1730223967.0
Automation
CORAL SHORES BEHAVIORAL HEALTH : 1366895096.0
Automation
CORDELL MEMORIAL HOSPITAL : 1750384426.0
Automation
CORDELL MEMORIAL HOSPITAL : 1750384426.0
Automation
CORNERSTONE HOSPITAL-BOSSIER CITY : 1912951112.0
Automation
CORNERSTONE HOSPITAL-BOSSIER CITY : 1912951112.0
Automation
CORNERSTONE HOSPITAL OF SOUTHEAST ARIZONA : 1043264278.0
Automation
CORNERSTONE HOSPITAL OF SOUTHEAST ARIZONA : 1043264278.0
Automation
CORNERSTONE HOSPITAL OF WEST MONROE : 1104870336.0
Automation
CORNERSTONE HOSPITAL OF WEST MONROE : 1104870336.0
Automation
CORNERSTONE REGIONAL HOSPITAL : 1386652527.0
Automation
CORNERSTONE REGIONAL HOSPITAL : 1386652527.0
Automation
CORONA REGIONAL MEDICAL CENTER : 1184088049.0
Automation
CORONA REGIONAL MEDICAL CENTER : 1184088049.0
Automation
COTTAGE HOSPITAL : 1811401656.0
Automation
COTTAGE HOSPITAL : 1811401656.0
Automation
COTTONWOOD SPRINGS : 1023403128.0
Automation


Automation
GWINNETT HOSPITAL SYSTEM, INC : 1619245107.0
Automation
GWINNETT HOSPITAL SYSTEM, INC : 1619245107.0
Automation
GEORGE WASHINGTON MEDICAL FACULTY ASSOCIATES : 1720228687.0
Automation
HACKENSACK UMC PALISADES : 1659571420.0
Automation
HACKENSACK UMC PALISADES : 1659571420.0
Automation
HACKENSACK UNIVERSITY MEDICAL CENTER AT PASCACK VALLEY : 1952749913.0
Automation
HACKENSACK UNIVERSITY MEDICAL CENTER AT PASCACK VALLEY : 1952749913.0
Automation
HAHNEMANN UNIVERSITY HOSPITAL : 1649650359.0
Automation
HAHNEMANN UNIVERSITY HOSPITAL : 1649650359.0
Automation
HALIFAX REGIONAL MEDICAL CENTER : 1346273943.0
Mismatched npis
HALIFAX REGIONAL MEDICAL CENTER : 1023336260.0
Automation
HAMILTON CENTER INC. : 1205980901.0
Automation
HAMILTON CENTER INC. : 1205980901.0
Automation
HAMILTON CENTER INC. : 1205980901.0
Automation
HAMILTON COUNTY HOSPITAL : 1235231697.0
Automation
HAMILTON COUNTY HOSPITAL : 1235231697.0
Automation
HAMILTON MEDICAL CENTER : 1083046684.0
Automation
HAMILTON MEDICAL

MERCY HOSPITAL : 1629078712.0
Automation
MERCY HOSPITAL : 1629078712.0
Automation
MERCY HOSPITAL & MEDICAL CTR. : 1730166224.0
MERCY HOSPITAL & MEDICAL CTR. : 1730166224.0
Automation
MERCY IOWA CITY : 1982664124.0
Automation
MERCY IOWA CITY : 1982664124.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER : 1417493073.0
Automation
MERCY MEDICAL CENTER INC. : 1477590198.0
Automation
MERCY MEDICAL CENTER INC. : 1477590198.0
Automation
MERCY TYLER HOSPITAL : 1023010261.0
Automation
MERCY TYLER HOSPIT

HS REHAB HOSP OF NORTH AL : 1215490842.0
Automation
REHABILITATION INSTITUTE OF CHICAGO : 1205879095.0
Automation
REID HOSPITAL & HEALTH CARE SERVICES : 1063457380.0
Automation
REID HOSPITAL & HEALTH CARE SERVICES : 1063457380.0
Automation
RENO BEHAVIORAL HEALTHCARE HOSPITAL : 1619488525.0
Automation
RENO BEHAVIORAL HEALTHCARE HOSPITAL : 1619488525.0
Automation
RENOWN REGIONAL MEDICAL CENTER : 1124098421.0
Automation
RENOWN REGIONAL MEDICAL CENTER : 1124098421.0
Automation
REYNOLDS ARMY COMMUNITY HOSPITAL : 1487752960.0
Automation
REYNOLDS ARMY COMMUNITY HOSPITAL : 1487752960.0
Automation
REYNOLDS MEMORIAL HOSPITAL INC : 1144228305.0
Automation
REYNOLDS MEMORIAL HOSPITAL INC : 1144228305.0
Automation
RICE MEMORIAL HOSPITAL : 1619064193.0
Automation
RICE MEMORIAL HOSPITAL : 1619064193.0
Automation
ROUDEBUSH VA MEDICAL CENTER : 1467836684.0
Automation
ROUDEBUSH VA MEDICAL CENTER : 1467836684.0
Automation
ROUDEBUSH VA MEDICAL CENTER : 1467836684.0
Automation
RICHLAND PARISH HOSPITAL : 165

THE CHRIST HOSPITAL : 1124577705.0
Automation
THE CRAIG AND FRANCES LINDNER CENTER OF HOPE : 1023207750.0
Automation
THE CRAIG AND FRANCES LINDNER CENTER OF HOPE : 1023207750.0
Automation
DEPARTMENT OF VETERANS AFFAIRS MEDICAL  CENTER : 1962748780.0
Automation
DEPARTMENT OF VETERANS AFFAIRS MEDICAL  CENTER : 1962748780.0
Automation
GEORGE WASHINGTON UNIV MEDICAL FAC ASSOCIATES, INC. : 1396973459.0
Automation
GRIFFIN HOSPITAL : 1184639973.0
GRIFFIN HOSPITAL : 1184639973.0
Automation
THE HEALTHCARE AUTHORITY OF THE CITY OF GREENVILLE - LV STABLER HOSPITAL : 1780655332.0
Automation
THE HOSPITAL OF CENTRAL CONNECTICUT : 1568532810.0
Automation
THE HOSPITAL OF CENTRAL CONNECTICUT : 1568532810.0
Automation
THE INSTITUTE FOR REHABILITATION AND RESEARCH : 1801016704.0
Automation
THE INSTITUTE FOR REHABILITATION AND RESEARCH : 1801016704.0
Automation
KING'S DAUGHTERS' HOSPITAL & HEALTH SERVICES : 1023232592.0
Automation
KING'S DAUGHTERS' HOSPITAL & HEALTH SERVICES : 1023232592.0
Automation
LOWE

None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None
None : None


In [157]:
xyz['NPI_NUMBER']=npis

In [155]:
npis

[1609859891.0,
 1609859891.0,
 1104029180.0,
 1104029180.0,
 1063449361.0,
 1063449361.0,
 1255300414.0,
 1255300414.0,
 1740331693.0,
 1740331693.0,
 1811992084.0,
 1811992084.0,
 1730304361.0,
 1730304361.0,
 1578911830.0,
 1578911830.0,
 1043216583.0,
 1043216583.0,
 1063737765.0,
 1063737765.0,
 1609942572.0,
 1609942572.0,
 1225103922.0,
 1225103922.0,
 1235392192.0,
 1235392192.0,
 1720191059.0,
 1851946909.0,
 1851946909.0,
 1902839673.0,
 1902839673.0,
 1902839673.0,
 1699709576.0,
 1699709576.0,
 1528398534.0,
 1528398534.0,
 1265908032.0,
 1265908032.0,
 1912039017.0,
 1912039017.0,
 1487650024.0,
 1023262441.0,
 1023262441.0,
 1609197979.0,
 1609197979.0,
 1346228541.0,
 1346228541.0,
 1891938122.0,
 1891938122.0,
 1730337460.0,
 1730337460.0,
 1821035940.0,
 1821035940.0,
 1326010273.0,
 1326010273.0,
 1427295849.0,
 1780134270.0,
 1285845537.0,
 1285845537.0,
 1619444353.0,
 1619444353.0,
 1841471356.0,
 1841471356.0,
 1265577191.0,
 1265577191.0,
 1134263254.0,
 113426325

In [162]:
customer_names=[]
for row in xyz.itertuples():
    if row.CUSTOMER_NAME_x=='None':
        customer_name = row.CUSTOMER_NAME_y.upper()
    else:
        customer_name = row.CUSTOMER_NAME_x
    customer_names.append(customer_name)

In [164]:
xyz['CUSTOMER_NAME']=customer_names

In [169]:
address_match = xyz[xyz.NPI_NUMBER=='Address']

In [173]:
npi_hosp['MAILING_ADDRESS_1']

5694        55 S RAYMOND AVE STE 105
23780               801 N BEDELL AVE
960          1345 PHILOMENA ST # 362
7799                 711 N TAYLOR ST
10835    2100 MACK BLVD, PO BOX 4000
                    ...             
3385              1584 TULLIE CIR NE
3383              3845 GREY ABBEY DR
11610        1935 SHORE PKWY APT 16B
11638                131 CHADWICK ST
20255                    1258 OAK ST
Name: MAILING_ADDRESS_1, Length: 20070, dtype: object

In [ ]:
dict_list_2 = []
count = 0
for row in address_match.itertuples():
    if customer in npi_hospitals:
        npi_name = customer
        high_score = 100
    else:
        high_score = 0
        for hospital in npi_hospitals:
            score = fuzz.ratio(hospital, customer)
            if score>90:
                high_score = score
                npi_name = hospital
                break
            elif score>high_score:
                high_score = score
                npi_name = hospital
    new_dict = {
        'CUSTOMER_NAME': customer,
        'NAME': npi_name,
        'SCORE': high_score
        }
    dict_list.append(new_dict)
    count += 1
    print(f'{customer} is {npi_name}, {high_score}')
    print(f'{count} customers matched')
    print('----')
    print('')

In [176]:
address_match.to_excel('../../Data/Credentialling/addressonly.xlsx')

In [177]:
npi = pd.read_csv('../../Data/NPI/npidata_pfile_20050523-20200112.csv')

KeyboardInterrupt: 

In [181]:
zyx = xyz[['CUSTOMER_NAME_x', 'CUSTOMER_KEY', 'CUSTOMER_NBR_x',
       'FULL_DT', 'ME', 'FIRST_NAME',
       'MIDDLE_NAME', 'LAST_NAME', 'NPI_NUMBER',
       'CUSTOMER_NAME']]

In [206]:
with_npi = pd.merge(zyx, npi_hosp, on='NPI')

In [186]:
npi_hosp['NPI']

5694     1124596382
23780    1104830900
960      1942795133
7799     1932109048
10835    1033759626
            ...    
3385     1619014156
3383     1740337153
11610    1225258452
11638    1023198041
20255    1568541084
Name: NPI, Length: 20070, dtype: int64

In [201]:
zyx = zyx[(zyx['NPI_NUMBER']!='None')&(zyx['NPI_NUMBER']!='Address')]
zyx['NPI'] = zyx['NPI_NUMBER'].astype(int)

In [203]:
xyz.columns

Index(['CUSTOMER_NAME_x', 'NAME', 'SCORE', 'NPI', 'Taxonomies', 'OTHER_NAME',
       'MAILING_ADDRESS_1', 'MAILING_ADDRESS_2', 'MAILING_CITY',
       'MAILING_STATE', 'MAIING_ZIP', 'MAILING_PHONE', 'LOCATION_ADDRESS_1',
       'LOCATION_ADDRESS_2', 'LOCATION_CITY', 'LOCATION_STATE', 'LOCATION_ZIP',
       'LOCATION_PHONE', 'LAST_UPDATE_DATE', 'CUSTOMER_KEY', 'CUSTOMER_NBR_x',
       'CUSTOMER_ISELL_LOGIN_x', 'CUSTOMER_TYPE_DESC_x', 'CUSTOMER_TYPE_x',
       'CUSTOMER_CATEGORY_DESC_x', 'CUSTOMER_NBR_y', 'CUSTOMER_ISELL_LOGIN_y',
       'CUSTOMER_NAME_y', 'CUSTOMER_TYPE_DESC_y', 'CUSTOMER_TYPE_y',
       'CUSTOMER_CATEGORY_DESC_y', 'FULL_DT', 'ME', 'FIRST_NAME',
       'MIDDLE_NAME', 'LAST_NAME', 'POLO_STATE', 'address', 'city', 'state',
       'zipcode', 'other name', 'npi', 'Google state', 'NPI_NUMBER',
       'CUSTOMER_NAME'],
      dtype='object')

In [204]:
PROD

,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,CUSTOMER_KEY,ORDER_PRODUCT_ID,PRODUCT_DESC,PRODUCT_ID
0,2020-05-05,01720732471,DAVID,LOREN,BIXLER,52982,4915513,Full AMA Physician Profile,4915513
1,2020-05-14,70416820030,FARRUKH,SHOHAB,KHAN,56701,4915513,Full AMA Physician Profile,4915513
2,2020-05-04,01103180135,MEAGAN,AMANDA,KELLY,56357,4915513,Full AMA Physician Profile,4915513
3,2020-05-26,04816180452,CONNOR,NaN,ERNST,56806,4915513,Full AMA Physician Profile,4915513
4,2020-05-15,00102000498,MELANIE,PIKE,NICHOLLS,134657,4915513,Full AMA Physician Profile,4915513
...,...,...,...,...,...,...,...,...,...
45996,2020-05-05,04814081848,ADEEB,JAY,DWAIRY,55514,4915502,Full Profile - No ABMS,4915502
45997,2020-05-29,04102922200,BONNIE,LOUISE,WRIGHT,55514,4915502,Full Profile - No ABMS,4915502
45998,2020-05-05,04706931789,KARL,EVERETTE,LAWRENCE,55514,4915502,Full Profile - No ABMS,4915502
45999,2020-05-16,03506161265,ANEELA,NaN,COX,55514,4915502,Full Profile - No ABMS,4915502


In [205]:
len(zyx)

5525

In [228]:
pd.merge(with_npi, PROD, on='CUSTOMER_KEY').to_excel('../../Data/Credentialling/may_profile_orders.xlsx', index=False)

In [209]:
len(PROD)

40923

In [213]:
hosps = list(HOSP_CUST.CUSTOMER_KEY)

In [216]:
PROD[PROD.CUSTOMER_KEY.isin(hosps)].drop_duplicates('CUSTOMER_KEY')

,FULL_DT,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,CUSTOMER_KEY,ORDER_PRODUCT_ID,PRODUCT_DESC,PRODUCT_ID
0,2020-05-05,01720732471,DAVID,LOREN,BIXLER,52982,4915513,Full AMA Physician Profile,4915513
1,2020-05-14,70416820030,FARRUKH,SHOHAB,KHAN,56701,4915513,Full AMA Physician Profile,4915513
5,2020-05-07,00102881524,CLIFTON,G,YOUNGBLOOD,121841,4915513,Full AMA Physician Profile,4915513
8,2020-05-19,01611140564,ROBERT,BARLING,LURVEY,60684,4915513,Full AMA Physician Profile,4915513
13,2020-05-19,00102961153,EDYTHE,WOODRUFF,STEWART,55290,4915513,Full AMA Physician Profile,4915513
...,...,...,...,...,...,...,...,...,...
43324,2020-05-15,04814090472,PHILIP,E,SCHERRER,52088,4915522,Reapp AMA Physician Pro + CM,4915522
43367,2020-05-25,00401110992,MARK,GERARD,VIEGAS,54576,4915522,Reapp AMA Physician Pro + CM,4915522
43389,2020-05-05,04816130757,ROYCE,PAUL,GRAY,58323,4915522,Reapp AMA Physician Pro + CM,4915522
43413,2020-05-20,01875111163,BECKY,ANN,SLATER,52463,4915522,Reapp AMA Physician Pro + CM,4915522


In [217]:
address_match = pd.read_excel('../../Data/Credentialling/addressonly.xlsx')

In [222]:
address_match = address_match[['CUSTOMER_KEY','address', 'city', 'state',
       'zipcode', 'other name', 'CUSTOMER_NAME']]

In [227]:
pd.merge(PROD, address_match, on='CUSTOMER_KEY').to_excel(('../../Data/Credentialling/address_match.xlsx'))

In [225]:
address_match['CUSTOMER_KEY']=address_match['CUSTOMER_KEY'].astype(str)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [230]:
x = pd.read_excel('../../Data/Credentialling/May_Credentialing_Product_20-06-25.xlsx')

In [231]:
x.groupby('CUSTOMER_KEY').count()

,CUSTOMER_NAME,NPI,MAILING_ADDRESS_1,MAILING_ADDRESS_2,MAILING_CITY,MAILING_STATE,MAIING_ZIP,FULL_DT_y,ME_y,FIRST_NAME_y,MIDDLE_NAME_y,LAST_NAME_y,PRODUCT_DESC
CUSTOMER_KEY,,,,,,,,,,,,,
51732,11,11,11,0,11,11,11,11,11,11,8,11,11
51739,5,5,5,5,5,5,5,5,5,5,4,5,5
51741,2,2,2,2,2,2,2,2,2,2,2,2,2
51742,2,2,2,0,2,2,2,2,2,2,2,2,2
51744,7,7,7,7,7,7,7,7,7,7,5,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136560,2,2,2,0,2,2,2,2,2,2,2,2,2
136578,1,1,1,1,1,1,1,1,1,1,1,1,1
136655,6,6,6,6,6,6,6,6,6,6,4,6,6
